In [1]:
import numpy as np 
import scipy as sp 
import matplotlib.pyplot as plt 
import pandas as pd 

#x = [T_m, T_c]
#u = [Q_in, T_env]
#theta = [C_m, C_c, hA_m2c, hA_m2env, hA_eff, mda]
chp_air = 1000 #J/kg/K

def RK4_solver(f, x0, t0, t1, N, u, theta):
    dt = (t1-t0)/N
    t = np.linspace(t0, t1, N)
    x_solved = np.zeros((N+1, len(x0)))
    dx_solved = np.zeros((N, len(x0)))
    x_solved[0] = x0
    for i in range(N): 
        K1 = f(x_solved[i], u[i], theta) #Make sure f returns an array of shape (2,)
        K2 = f(x_solved[i]+K1/2, u[i], theta)
        K3 = f(x_solved[i]+K2/2, u[i], theta)
        K4 = f(x_solved[i]+K3, u[i], theta)
        K = K1/6 + K2/3 + K3/3 + K4/6
        x_solved[i+1] = x_solved[i] + K*dt
        dx_solved[i] = K
    return x_solved[:-1], dx_solved

def dxdt(x, u, theta): 
    Q_HE = theta[5]*chp_air*(1-np.exp(-theta[4]/(theta[5]*chp_air)))*(x[1]-u[1])
    dTm_dt = (u[0]-theta[2]*(x[0]-x[1])-theta[3]*(x[0]-u[1]))/theta[0]
    dTc_dt = (theta[2]*(x[0]-x[1])-Q_HE)/theta[1]
    return np.array([dTm_dt, dTc_dt])

def find_input_vals(u_range, N, change=0.01):
    #u_range = [(u1_min, u1_max), (u2_min, u2_max)]
    u1 = np.zeros(N)
    u2 = np.zeros(N)
    u1_rand = np.random.uniform(u_range[0][0], u_range[0][1])
    u2_rand = np.random.uniform(u_range[1][0], u_range[1][1])
    for i in range(0, N): 
        ch = np.random.uniform(0, 1)
        u1[i] = u1_rand
        u2[i] = u2_rand
        if ch < change: 
            u1_rand = np.random.uniform(u_range[0][0], u_range[0][1])
            #u2_rand = np.random.uniform(u_range[1][0], u_range[1][1])
            
    return np.array([u1, u2]).T
    


In [2]:
t0 = 0 
t1 = 60 #s 
N = 600
t = np.linspace(t0, t1, N)
N_sim = 10000
jump_el = 5 #How many steps before storing a value as NN input

x0_range = [(10, 50), (10, 50)] #T_m0 = T_c0
u_range = [(0, 50), (10, 30)]
theta_range = [(100, 800), (40, 400), (5,20), (0.1,1), (1,5), (0.3, 1.2)]

X_NN = np.zeros((N_sim*int(N/jump_el), 8)) #NN 8 inputs: [x1, x1, dx1, dx2, u1, u2, Cm, Cc]
y_NN = np.zeros((N_sim*int(N/jump_el), 6)) #NN 6 outputs (all parameters also considering Cm and Cc)

for i in range(N_sim): 
    print("Simulation: {}".format(i))
    theta = np.array([np.random.uniform(th[0], th[1]) for th in theta_range])
    #Find two values to oscillate between for inputs    
    x0 = np.random.uniform(x0_range[0][0], x0_range[0][1])*np.ones(2)
    u_in = find_input_vals(u_range, N, change=0.005)
    x_sol, dx_sol = RK4_solver(dxdt, x0, t0, t1, N, u_in, theta)
    
    #Storing values in arrays
    x_sol = x_sol[::jump_el]
    dx_sol = dx_sol[::jump_el]
    u_in = u_in[::jump_el]
    j = 0
    for x, dx, u in zip(x_sol, dx_sol, u_in): 
        X_NN[i*len(x_sol)-1+j][0] = x[0]
        X_NN[i*len(x_sol)-1+j][1] = x[1]
        X_NN[i*len(x_sol)-1+j][2] = dx[0]
        X_NN[i*len(x_sol)-1+j][3] = dx[1]
        X_NN[i*len(x_sol)-1+j][4] = u[0]
        X_NN[i*len(x_sol)-1+j][5] = u[1]
        X_NN[i*len(x_sol)-1+j][6] = theta[0]
        X_NN[i*len(x_sol)-1+j][7] = theta[1]
        
        y_NN[i*len(x_sol)-1+j][0] = theta[0]
        y_NN[i*len(x_sol)-1+j][1] = theta[1]
        y_NN[i*len(x_sol)-1+j][2] = theta[2]
        y_NN[i*len(x_sol)-1+j][3] = theta[3]
        y_NN[i*len(x_sol)-1+j][4] = theta[4]
        y_NN[i*len(x_sol)-1+j][5] = theta[5]
        j += 1
#X_NN = X_NN.reshape(-1, 8, 1)

Simulation: 0
Simulation: 1
Simulation: 2
Simulation: 3
Simulation: 4
Simulation: 5
Simulation: 6
Simulation: 7
Simulation: 8
Simulation: 9
Simulation: 10
Simulation: 11
Simulation: 12
Simulation: 13
Simulation: 14
Simulation: 15
Simulation: 16
Simulation: 17
Simulation: 18
Simulation: 19
Simulation: 20
Simulation: 21
Simulation: 22
Simulation: 23
Simulation: 24
Simulation: 25
Simulation: 26
Simulation: 27
Simulation: 28
Simulation: 29
Simulation: 30
Simulation: 31
Simulation: 32
Simulation: 33
Simulation: 34
Simulation: 35
Simulation: 36
Simulation: 37
Simulation: 38
Simulation: 39
Simulation: 40
Simulation: 41
Simulation: 42
Simulation: 43
Simulation: 44
Simulation: 45
Simulation: 46
Simulation: 47
Simulation: 48
Simulation: 49
Simulation: 50
Simulation: 51
Simulation: 52
Simulation: 53
Simulation: 54
Simulation: 55
Simulation: 56
Simulation: 57
Simulation: 58
Simulation: 59
Simulation: 60
Simulation: 61
Simulation: 62
Simulation: 63
Simulation: 64
Simulation: 65
Simulation: 66
Simul

Simulation: 520
Simulation: 521
Simulation: 522
Simulation: 523
Simulation: 524
Simulation: 525
Simulation: 526
Simulation: 527
Simulation: 528
Simulation: 529
Simulation: 530
Simulation: 531
Simulation: 532
Simulation: 533
Simulation: 534
Simulation: 535
Simulation: 536
Simulation: 537
Simulation: 538
Simulation: 539
Simulation: 540
Simulation: 541
Simulation: 542
Simulation: 543
Simulation: 544
Simulation: 545
Simulation: 546
Simulation: 547
Simulation: 548
Simulation: 549
Simulation: 550
Simulation: 551
Simulation: 552
Simulation: 553
Simulation: 554
Simulation: 555
Simulation: 556
Simulation: 557
Simulation: 558
Simulation: 559
Simulation: 560
Simulation: 561
Simulation: 562
Simulation: 563
Simulation: 564
Simulation: 565
Simulation: 566
Simulation: 567
Simulation: 568
Simulation: 569
Simulation: 570
Simulation: 571
Simulation: 572
Simulation: 573
Simulation: 574
Simulation: 575
Simulation: 576
Simulation: 577
Simulation: 578
Simulation: 579
Simulation: 580
Simulation: 581
Simulati

Simulation: 1033
Simulation: 1034
Simulation: 1035
Simulation: 1036
Simulation: 1037
Simulation: 1038
Simulation: 1039
Simulation: 1040
Simulation: 1041
Simulation: 1042
Simulation: 1043
Simulation: 1044
Simulation: 1045
Simulation: 1046
Simulation: 1047
Simulation: 1048
Simulation: 1049
Simulation: 1050
Simulation: 1051
Simulation: 1052
Simulation: 1053
Simulation: 1054
Simulation: 1055
Simulation: 1056
Simulation: 1057
Simulation: 1058
Simulation: 1059
Simulation: 1060
Simulation: 1061
Simulation: 1062
Simulation: 1063
Simulation: 1064
Simulation: 1065
Simulation: 1066
Simulation: 1067
Simulation: 1068
Simulation: 1069
Simulation: 1070
Simulation: 1071
Simulation: 1072
Simulation: 1073
Simulation: 1074
Simulation: 1075
Simulation: 1076
Simulation: 1077
Simulation: 1078
Simulation: 1079
Simulation: 1080
Simulation: 1081
Simulation: 1082
Simulation: 1083
Simulation: 1084
Simulation: 1085
Simulation: 1086
Simulation: 1087
Simulation: 1088
Simulation: 1089
Simulation: 1090
Simulation: 10

Simulation: 1516
Simulation: 1517
Simulation: 1518
Simulation: 1519
Simulation: 1520
Simulation: 1521
Simulation: 1522
Simulation: 1523
Simulation: 1524
Simulation: 1525
Simulation: 1526
Simulation: 1527
Simulation: 1528
Simulation: 1529
Simulation: 1530
Simulation: 1531
Simulation: 1532
Simulation: 1533
Simulation: 1534
Simulation: 1535
Simulation: 1536
Simulation: 1537
Simulation: 1538
Simulation: 1539
Simulation: 1540
Simulation: 1541
Simulation: 1542
Simulation: 1543
Simulation: 1544
Simulation: 1545
Simulation: 1546
Simulation: 1547
Simulation: 1548
Simulation: 1549
Simulation: 1550
Simulation: 1551
Simulation: 1552
Simulation: 1553
Simulation: 1554
Simulation: 1555
Simulation: 1556
Simulation: 1557
Simulation: 1558
Simulation: 1559
Simulation: 1560
Simulation: 1561
Simulation: 1562
Simulation: 1563
Simulation: 1564
Simulation: 1565
Simulation: 1566
Simulation: 1567
Simulation: 1568
Simulation: 1569
Simulation: 1570
Simulation: 1571
Simulation: 1572
Simulation: 1573
Simulation: 15

Simulation: 2000
Simulation: 2001
Simulation: 2002
Simulation: 2003
Simulation: 2004
Simulation: 2005
Simulation: 2006
Simulation: 2007
Simulation: 2008
Simulation: 2009
Simulation: 2010
Simulation: 2011
Simulation: 2012
Simulation: 2013
Simulation: 2014
Simulation: 2015
Simulation: 2016
Simulation: 2017
Simulation: 2018
Simulation: 2019
Simulation: 2020
Simulation: 2021
Simulation: 2022
Simulation: 2023
Simulation: 2024
Simulation: 2025
Simulation: 2026
Simulation: 2027
Simulation: 2028
Simulation: 2029
Simulation: 2030
Simulation: 2031
Simulation: 2032
Simulation: 2033
Simulation: 2034
Simulation: 2035
Simulation: 2036
Simulation: 2037
Simulation: 2038
Simulation: 2039
Simulation: 2040
Simulation: 2041
Simulation: 2042
Simulation: 2043
Simulation: 2044
Simulation: 2045
Simulation: 2046
Simulation: 2047
Simulation: 2048
Simulation: 2049
Simulation: 2050
Simulation: 2051
Simulation: 2052
Simulation: 2053
Simulation: 2054
Simulation: 2055
Simulation: 2056
Simulation: 2057
Simulation: 20

...

Simulation: 7796
Simulation: 7797
Simulation: 7798
Simulation: 7799
Simulation: 7800
Simulation: 7801
Simulation: 7802
Simulation: 7803
Simulation: 7804
Simulation: 7805
Simulation: 7806
Simulation: 7807
Simulation: 7808
Simulation: 7809
Simulation: 7810
Simulation: 7811
Simulation: 7812
Simulation: 7813
Simulation: 7814
Simulation: 7815
Simulation: 7816
Simulation: 7817
Simulation: 7818
Simulation: 7819
Simulation: 7820
Simulation: 7821
Simulation: 7822
Simulation: 7823
Simulation: 7824
Simulation: 7825
Simulation: 7826
Simulation: 7827
Simulation: 7828
Simulation: 7829
Simulation: 7830
Simulation: 7831
Simulation: 7832
Simulation: 7833
Simulation: 7834
Simulation: 7835
Simulation: 7836
Simulation: 7837
Simulation: 7838
Simulation: 7839
Simulation: 7840
Simulation: 7841
Simulation: 7842
Simulation: 7843
Simulation: 7844
Simulation: 7845
Simulation: 7846
Simulation: 7847
Simulation: 7848
Simulation: 7849
Simulation: 7850
Simulation: 7851
Simulation: 7852
Simulation: 7853
Simulation: 78

Simulation: 8278
Simulation: 8279
Simulation: 8280
Simulation: 8281
Simulation: 8282
Simulation: 8283
Simulation: 8284
Simulation: 8285
Simulation: 8286
Simulation: 8287
Simulation: 8288
Simulation: 8289
Simulation: 8290
Simulation: 8291
Simulation: 8292
Simulation: 8293
Simulation: 8294
Simulation: 8295
Simulation: 8296
Simulation: 8297
Simulation: 8298
Simulation: 8299
Simulation: 8300
Simulation: 8301
Simulation: 8302
Simulation: 8303
Simulation: 8304
Simulation: 8305
Simulation: 8306
Simulation: 8307
Simulation: 8308
Simulation: 8309
Simulation: 8310
Simulation: 8311
Simulation: 8312
Simulation: 8313
Simulation: 8314
Simulation: 8315
Simulation: 8316
Simulation: 8317
Simulation: 8318
Simulation: 8319
Simulation: 8320
Simulation: 8321
Simulation: 8322
Simulation: 8323
Simulation: 8324
Simulation: 8325
Simulation: 8326
Simulation: 8327
Simulation: 8328
Simulation: 8329
Simulation: 8330
Simulation: 8331
Simulation: 8332
Simulation: 8333
Simulation: 8334
Simulation: 8335
Simulation: 83

Simulation: 8763
Simulation: 8764
Simulation: 8765
Simulation: 8766
Simulation: 8767
Simulation: 8768
Simulation: 8769
Simulation: 8770
Simulation: 8771
Simulation: 8772
Simulation: 8773
Simulation: 8774
Simulation: 8775
Simulation: 8776
Simulation: 8777
Simulation: 8778
Simulation: 8779
Simulation: 8780
Simulation: 8781
Simulation: 8782
Simulation: 8783
Simulation: 8784
Simulation: 8785
Simulation: 8786
Simulation: 8787
Simulation: 8788
Simulation: 8789
Simulation: 8790
Simulation: 8791
Simulation: 8792
Simulation: 8793
Simulation: 8794
Simulation: 8795
Simulation: 8796
Simulation: 8797
Simulation: 8798
Simulation: 8799
Simulation: 8800
Simulation: 8801
Simulation: 8802
Simulation: 8803
Simulation: 8804
Simulation: 8805
Simulation: 8806
Simulation: 8807
Simulation: 8808
Simulation: 8809
Simulation: 8810
Simulation: 8811
Simulation: 8812
Simulation: 8813
Simulation: 8814
Simulation: 8815
Simulation: 8816
Simulation: 8817
Simulation: 8818
Simulation: 8819
Simulation: 8820
Simulation: 88

Simulation: 9247
Simulation: 9248
Simulation: 9249
Simulation: 9250
Simulation: 9251
Simulation: 9252
Simulation: 9253
Simulation: 9254
Simulation: 9255
Simulation: 9256
Simulation: 9257
Simulation: 9258
Simulation: 9259
Simulation: 9260
Simulation: 9261
Simulation: 9262
Simulation: 9263
Simulation: 9264
Simulation: 9265
Simulation: 9266
Simulation: 9267
Simulation: 9268
Simulation: 9269
Simulation: 9270
Simulation: 9271
Simulation: 9272
Simulation: 9273
Simulation: 9274
Simulation: 9275
Simulation: 9276
Simulation: 9277
Simulation: 9278
Simulation: 9279
Simulation: 9280
Simulation: 9281
Simulation: 9282
Simulation: 9283
Simulation: 9284
Simulation: 9285
Simulation: 9286
Simulation: 9287
Simulation: 9288
Simulation: 9289
Simulation: 9290
Simulation: 9291
Simulation: 9292
Simulation: 9293
Simulation: 9294
Simulation: 9295
Simulation: 9296
Simulation: 9297
Simulation: 9298
Simulation: 9299
Simulation: 9300
Simulation: 9301
Simulation: 9302
Simulation: 9303
Simulation: 9304
Simulation: 93

Simulation: 9729
Simulation: 9730
Simulation: 9731
Simulation: 9732
Simulation: 9733
Simulation: 9734
Simulation: 9735
Simulation: 9736
Simulation: 9737
Simulation: 9738
Simulation: 9739
Simulation: 9740
Simulation: 9741
Simulation: 9742
Simulation: 9743
Simulation: 9744
Simulation: 9745
Simulation: 9746
Simulation: 9747
Simulation: 9748
Simulation: 9749
Simulation: 9750
Simulation: 9751
Simulation: 9752
Simulation: 9753
Simulation: 9754
Simulation: 9755
Simulation: 9756
Simulation: 9757
Simulation: 9758
Simulation: 9759
Simulation: 9760
Simulation: 9761
Simulation: 9762
Simulation: 9763
Simulation: 9764
Simulation: 9765
Simulation: 9766
Simulation: 9767
Simulation: 9768
Simulation: 9769
Simulation: 9770
Simulation: 9771
Simulation: 9772
Simulation: 9773
Simulation: 9774
Simulation: 9775
Simulation: 9776
Simulation: 9777
Simulation: 9778
Simulation: 9779
Simulation: 9780
Simulation: 9781
Simulation: 9782
Simulation: 9783
Simulation: 9784
Simulation: 9785
Simulation: 9786
Simulation: 97

In [3]:
np.save("X_NN_case1",X_NN)
np.save("y_NN_case1", y_NN)

In [361]:
X_NN_loaded = np.load("X_NN_case1.npy")
y_NN_loaded = np.load("y_NN_case1.npy")

In [363]:
y_NN_loaded.shape

(120000, 6)